# Forsgren 3.4: The effect of the barrier parameter
(Taken from Forsgren A, et al. Interior methods for nonlinear optimization. SIAM review. 2002;44(4):525-97)

> **Requirements**
> - python3.X
> - pyomo 5.X.X
> - [ipopt](https://github.com/coin-or/Ipopt) 3.12.12 (tested with)

how to get the ipopt output!

### 0. Introduction

In this problem, the effects of the nonlinearities on some optimization algorithms (interior point) is showed.  
For this the following *nonlinear* optimization problem is considered:

\begin{align}
\text{minimize} \quad & \dfrac{10}{3}x_{1} x_{2} + \dfrac{1}{6} x_{1} \\
\text{subject to} \quad & \dfrac{19}{16} - x^{2}_{1} - \dfrac{5}{2}x_{2}^{2} \geq 0, \\
& x_{1} - x_{2} + \dfrac{3}{5} \geq 0
\end{align}

In the following figure the feasible region is displayed: inside the elipse (first constraint) and below the dotted line (second constraint). Furthermore, the contours of the *barrier* objective function are shown.

![Forsgren](./img/forsgren1.png)  
Figure: The feasible region and objective function level-curves (Credit: Forsgren et al 2002)

Clearly, if *ipopt* is used as a solver, the initial guess will be essential when solving the problem.

### 1. Package imports and model declaration

As usual the pyomo environment has to be imported first.

In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
m = ConcreteModel()  #: concrete model

### 2. Variables

In this problem there are two variables. A `Set` object is not necessary, though it could be used to create an indexed variable `x`.  
Since, this problem is nonlinear, it is *necessary* to specify an initial guess. In pyomo, this is usually done at construction level using the `initialize` keyword from the constructor.  
For a simple variable (not indexed) a floating point number is sufficient.

In [2]:
m.x1 = Var(initialize=0)  #: same as no initial guess
m.x2 = Var(initialize=0)  #: same as no initial guess

However, if the variable is not initialed at the construction, its initial value often set to $\max (0, x_{lb})$. This is problematic in several situations as the local linearizations of the constraints might not define a well-posed problem. 

### 3. Constraints and Objective

Singleton constraints can have their respective expressions declared directly at construction. This is done with the `expr` keyword.

In [3]:
m.c1 = Constraint(expr=19/16 - m.x1 ** 2 - 5/2 * m.x2 ** 2 >= 0)
m.c2 = Constraint(expr=m.x1 - m.x2 + 3/5 >= 0)

Similarly, the objective function is constructed.

In [4]:
m.of = Objective(expr=10/3 * m.x1 * m.x2 + 1/6 * m.x1, sense=minimize)

### 4. Solution and discussion

To solve this problem, *ipopt* is used. *ipopt* is an interior-point optimization algorithm. It solves a sequence of problems called *barrier* subproblems, in which the variable bounds are moved into the objective function.  
In pyomo, it is possible to turn off the solver output using the keyword `tee` of the `solve()` method. Also, note that the status of the solver is returned as a result of the solver call.

In [10]:
opt = SolverFactory('ipopt')
res = opt.solve(m, tee=True)
print(res['Solver'])   #: get the status after solver call

/tmp/tmpfzqznvlz.pyomo.nl -AMPL 
Ipopt vanilla0: 
This is Ipopt version vanilla0, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        2
        inequality constraints with only lower bounds:        2
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -1.1250000e+00 0.00e+00 3.20

One way to show the results is by means of `.display()` method for variables.  

In [6]:
m.x1.display()
m.x2.display()

x1 : Size=1, Index=None
    Key  : Lower : Value               : Upper : Fixed : Stale : Domain
    None :  None : -0.3250000041542386 :  None : False : False :  Reals
x2 : Size=1, Index=None
    Key  : Lower : Value               : Upper : Fixed : Stale : Domain
    None :  None : 0.27500000354017506 :  None : False : False :  Reals


*ipopt* found a *local* solution. From the Figure at the introduction section, it is clear that if the initial guess is perturbed; the results might change.  
In order to test this issue, one can set the initial guess to a different value. For a singleton variable this is done with the `set_value()` method.  
`set_value()` will have different behaviour if the variable is indexed.

In [7]:
m.x1.set_value(1)  #: Change the initial guesses            
m.x2.set_value(-1)

Then the model can be resolved:

In [8]:
res = opt.solve(m, tee=False)
print(res['Solver'])


- Status: ok
  Message: Ipopt vanilla0\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.01589798927307129



And the results displayed in similar way.

In [9]:
m.x1.display()
m.x2.display()

x1 : Size=1, Index=None
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
    None :  None : 0.7500000031123425 :  None : False : False :  Reals
x2 : Size=1, Index=None
    Key  : Lower : Value               : Upper : Fixed : Stale : Domain
    None :  None : -0.5000000018801641 :  None : False : False :  Reals


Note that the results have changed. This illustrates the issue of providing *good* initial guesses for nonlinear problems.  
The real solution of this problem is in fact $x^{*} = (\dfrac{3}{4}, \dfrac{-1}{2})$.

### Credits:
- David Thierry (Carnegie Mellon University 2019)